In [47]:
from skimage.transform import warp, AffineTransform, ProjectiveTransform
from skimage.exposure import equalize_adapthist, equalize_hist, rescale_intensity, adjust_gamma, adjust_log, adjust_sigmoid
from skimage.filters import gaussian
from skimage.util import random_noise
import random
import pylab as pl
from glob import glob
import warnings
from tqdm import tqdm
from PIL import Image
import scipy
warnings.simplefilter('ignore')

In [48]:
def randRange(a, b):
    '''
    a utility functio to generate random float values in desired range
    '''
    return pl.rand() * (b - a) + a


def randomAffine(im):
    '''
    wrapper of Affine transformation with random scale, rotation, shear and translation parameters
    '''
    tform = AffineTransform(scale=(randRange(0.75, 1.3), randRange(0.75, 1.3)),
                            rotation=randRange(-0.25, 0.25),
                            shear=randRange(-0.2, 0.2),
                            translation=(randRange(-im.shape[0]//10, im.shape[0]//10), 
                                         randRange(-im.shape[1]//10, im.shape[1]//10)))
    return warp(im, tform.inverse, mode='reflect')


def randomPerspective(im):
    '''
    wrapper of Projective (or perspective) transform, from 4 random points selected from 4 corners of the image within a defined region.
    '''
    region = 1/4
    A = pl.array([[0, 0], [0, im.shape[0]], [im.shape[1], im.shape[0]], [im.shape[1], 0]])
    B = pl.array([[int(randRange(0, im.shape[1] * region)), int(randRange(0, im.shape[0] * region))], 
                  [int(randRange(0, im.shape[1] * region)), int(randRange(im.shape[0] * (1-region), im.shape[0]))], 
                  [int(randRange(im.shape[1] * (1-region), im.shape[1])), int(randRange(im.shape[0] * (1-region), im.shape[0]))], 
                  [int(randRange(im.shape[1] * (1-region), im.shape[1])), int(randRange(0, im.shape[0] * region))], 
                 ])

    pt = ProjectiveTransform()
    pt.estimate(A, B)
    return warp(im, pt, output_shape=im.shape[:2])


def randomCrop(im):
    '''
    croping the image in the center from a random margin from the borders
    '''
    margin = 1/10
    start = [int(randRange(0, im.shape[0] * margin)),
             int(randRange(0, im.shape[1] * margin))]
    end = [int(randRange(im.shape[0] * (1-margin), im.shape[0])), 
           int(randRange(im.shape[1] * (1-margin), im.shape[1]))]
    return im[start[0]:end[0], start[1]:end[1]]


def randomIntensity(im):
    '''
    rescales the intesity of the image to random interval of image intensity distribution
    '''
    return rescale_intensity(im,
                             in_range=tuple(pl.percentile(im, (randRange(0,10), randRange(90,100)))),
                             out_range=tuple(pl.percentile(im, (randRange(0,10), randRange(90,100)))))

def randomGamma(im):
    '''
    Gamma filter for contrast adjustment with random gamma value.
    '''
    return adjust_gamma(im, gamma=randRange(0.5, 1.5))

def randomGaussian(im):
    '''
    Gaussian filter for bluring the image with random variance.
    '''
    return gaussian(im, sigma=randRange(0, 5))
    
def randomFilter(im):
    '''
    randomly selects an exposure filter from histogram equalizers, contrast adjustments, and intensity rescaler and applys it on the input image.
    filters include: equalize_adapthist, equalize_hist, rescale_intensity, adjust_gamma, adjust_log, adjust_sigmoid, gaussian
    '''
    Filters = [equalize_adapthist, equalize_hist, adjust_log, adjust_sigmoid, randomGamma, randomGaussian, randomIntensity]
    filt = random.choice(Filters)
    return filt(im)


def randomNoise(im):
    '''
    random gaussian noise with random variance.
    '''
    var = randRange(0.001, 0.01)
    return random_noise(im, var=var)
    

def augment(im, Steps=[randomGamma, randomPerspective, randomFilter]):
    '''
    image augmentation by doing a sereis of transfomations on the image.
    '''
    for step in Steps:
        im = step(im)
    return im

In [52]:
images = glob('./data_for_pytorch//train/*/*jpg')

In [53]:
len(images)

31

In [54]:
for image in tqdm(images):
    img = pl.imread(image)
    augm = [augment(img) for i in range(10)]
    body = image.split('/')[-1][:-4]
    for j, augmented_image in enumerate(augm):
        new_image_name = image.replace(body, body+'augmented'+str(j))
        scipy.misc.imsave(arr = augmented_image, name = new_image_name)




  0%|          | 0/31 [00:00<?, ?it/s]


  3%|▎         | 1/31 [00:00<00:06,  4.63it/s]


  6%|▋         | 2/31 [00:00<00:06,  4.68it/s]


 10%|▉         | 3/31 [00:00<00:05,  4.85it/s]


 13%|█▎        | 4/31 [00:00<00:05,  4.81it/s]


 16%|█▌        | 5/31 [00:01<00:05,  5.00it/s]


 19%|█▉        | 6/31 [00:01<00:05,  4.99it/s]


 23%|██▎       | 7/31 [00:01<00:04,  5.02it/s]


 26%|██▌       | 8/31 [00:01<00:04,  5.01it/s]


 29%|██▉       | 9/31 [00:01<00:04,  5.04it/s]


 32%|███▏      | 10/31 [00:02<00:04,  4.97it/s]


 35%|███▌      | 11/31 [00:02<00:04,  4.97it/s]


 39%|███▊      | 12/31 [00:02<00:03,  5.04it/s]


 42%|████▏     | 13/31 [00:02<00:03,  5.10it/s]


 45%|████▌     | 14/31 [00:02<00:03,  4.93it/s]


 48%|████▊     | 15/31 [00:03<00:03,  5.01it/s]


 52%|█████▏    | 16/31 [00:03<00:02,  5.24it/s]


 55%|█████▍    | 17/31 [00:03<00:02,  5.20it/s]


 58%|█████▊    | 18/31 [00:03<00:02,  5.13it/s]


 61%|██████▏   | 19/31 [00:03<00:02,  5.17it/s]


 65%|██████▍   